In [1]:
# PREPROCESSING

# Load joined data.

# Dimensionality reduction (removing columns).
    # Missing values ratio.

# Manual dimension reduction (dependent columns).
    # Reduce three gender columns to (all) and a (pct women).
    # Reduce Apps (all) / Offers (all) / Freshmen (all) to Offer Rate (all) and Yield Rate (all).
    # Drop redundant columns from dataframe.

# Validate data:
    # Find columns that should be pct_cols.
        # Check if pct_cols > 1 or < 0.
        # "Handle" this invalid data (clipping? removal?)
    # Define range_cols and that should be restricted to a range and ranges:
        # Check if range_cols in ranges.
        # "Handle" invalid data (clipping? removal?)
        
# Normalize continuous data.
    # Box-Cox transform all cols.
    # Normalize all columns (range 0 - 1).

# One-hot encode categorical data.

# Further dimensionality reduction: 
    # Low-variance filter.
    # High-correlation filter.
        # msno.heatmap()
    
# Impute missing values on remaining columns.
    # MICE for missing continuous values using linear regression plus errors.
    # kNN (?) for missing categorical values.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

JOINED_CSV_PATH = 'data/joined.csv'
NULL_THRESHOLD = 0.7

df = pd.read_csv(JOINED_CSV_PATH, index_col = 'SchoolId')

In [3]:
num_cols = df.select_dtypes('float').columns
null_freqs = df[num_cols].isna().sum() / len(df)
null_cols = num_cols.where(null_freqs > NULL_THRESHOLD).dropna()

df = df.drop(columns = null_cols)

In [4]:
df['Applications (women pct of all)'] = \
    df['Applications (women)'] / df['Applications (all)']
    
df['Offers (women pct of all)'] = \
    df['Offers (women)'] / df['Offers (all)']

df['Freshmen Enrolled (women pct of all)'] = \
    df['Freshmen Enrolled (women)'] / df['Freshmen Enrolled (all)']
    
df['Students (all)'] = \
    df['All Undergraduates'] + df['All Graduate Students']

df['Students (undergraduate pct of all)'] = \
    df['All Undergraduates'] / df['Students (all)']
    
df['Students (full-time pct of all undergrads)'] = \
    df['Full-Time Undergraduates'] / df['All Undergraduates']
    
drop_cols = ['Applications (women)', 'Applications (men)', 'Offers (all)',
             'Offers (women)', 'Offers (men)', 'Freshmen Enrolled (all)',
             'Freshmen Enrolled (women)', 'Freshmen Enrolled (men)',
             'Offer Rate (men)', 'Offer Rate (women)', 'Yield Rate (men)',
             'Yield Rate (women)', 'Undergraduates (women)',
             'Undergraduates (men)', 'Undergraduates (men pct of all)',
             'All Undergraduates', 'All Graduate Students',
             'Full-Time Undergraduates']

df = df.drop(columns = drop_cols)

In [29]:
num_cols = df.select_dtypes('float').columns

pct_cols = num_cols[df[num_cols].mean().between(0, 1)]
other_num_cols = [col for col in num_cols if col not in pct_cols]

invalid_vals = (df[pct_cols] < 0) | (df[pct_cols] > 1)

df = df.mask(invalid_vals)